In [1]:
import os
import zipfile
import shutil
from pathlib import Path
import yaml
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob

In [2]:
def setup_directories():
    """필요한 디렉토리 구조 생성"""
    dirs = [
        './dataset/images/train',
        './dataset/images/val', 
        './dataset/labels/train',
        './dataset/labels/val',
        'have_to_train_txt'
    ]
    
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
    print("디렉토리 구조 생성 완료")

In [3]:
# def extract_zip_files():
#     """압축 파일들 압축 해제"""
#     zip_files = {
#         'consolidated_images.zip': 'temp_images',
#         'consolidated_txt.zip': 'temp_labels',
#         'have_to_train_images.zip': 'inference_images'
#     }
    
#     for zip_file, extract_dir in zip_files.items():
#         if os.path.exists(zip_file):
#             with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#                 zip_ref.extractall(extract_dir)
#             print(f"{zip_file} 압축 해제 완료 -> {extract_dir}")
#         else:
#             print(f"경고: {zip_file} 파일을 찾을 수 없습니다.")

In [4]:
def prepare_train_val_split(test_size=0.2, random_state=42):
    """학습/검증 데이터 분할 및 정리"""
    # 이미지 파일 목록 가져오기
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(glob.glob(f'temp_images/{ext}'))
        image_files.extend(glob.glob(f'temp_images/{ext.upper()}'))
    
    print(f"총 {len(image_files)}개의 이미지 발견")
    
    # train/val 분할
    train_images, val_images = train_test_split(
        image_files, test_size=test_size, random_state=random_state
    )
    
    # 파일 복사 함수
    def copy_files(file_list, img_dest, label_dest):
        for img_path in file_list:
            img_name = os.path.basename(img_path)
            base_name = os.path.splitext(img_name)[0]
            
            # 이미지 복사
            shutil.copy2(img_path, os.path.join(img_dest, img_name))
            
            # 라벨 파일 찾기 및 복사
            label_path = f'temp_labels/{base_name}.txt'
            if os.path.exists(label_path):
                shutil.copy2(label_path, os.path.join(label_dest, f'{base_name}.txt'))
            else:
                print(f"경고: {base_name}.txt 라벨 파일을 찾을 수 없습니다.")
    
    # train 데이터 복사
    copy_files(train_images, 'dataset/images/train', 'dataset/labels/train')
    print(f"학습 데이터: {len(train_images)}개")
    
    # val 데이터 복사  
    copy_files(val_images, 'dataset/images/val', 'dataset/labels/val')
    print(f"검증 데이터: {len(val_images)}개")


In [5]:
def create_data_yaml():
    """YOLO 학습용 data.yaml 파일 생성"""
    data_config = {
        'train': 'dataset/images/train',
        'val': 'dataset/images/val',
        'nc': 2,
        'names': ['urine_strip', 'test_pad']
    }
    
    with open('data.yaml', 'w') as f:
        yaml.dump(data_config, f)
    
    print("data.yaml 파일 생성 완료")

In [6]:
print("=== 데이터 준비 시작 ===")
setup_directories()
# extract_zip_files()
print("=== 데이터 준비 완료 ===\n")

=== 데이터 준비 시작 ===
디렉토리 구조 생성 완료
=== 데이터 준비 완료 ===



In [7]:
prepare_train_val_split()
create_data_yaml()

총 1343개의 이미지 발견
학습 데이터: 1074개
검증 데이터: 269개
data.yaml 파일 생성 완료


In [8]:
# def train_yolo_model(epochs=100, imgsz=640, batch=16):
#     """YOLO 모델 학습"""
#     print("=== YOLO 모델 학습 시작 ===")
    
#     # YOLOv8 모델 로드 (nano 버전, 빠른 학습용)
#     model = YOLO('yolov8s.pt')
#     # 수정 코드 (기존 best.pt 로드)
#     # model = YOLO('runs/detect2/urine_strip_train/weights/best.pt')
    
#     # 학습 실행
#     results = model.train(
#         data='data.yaml',
#         epochs=epochs,
#         imgsz=imgsz,
#         batch=batch,
#         save=True,
#         project='runs/detect2',
#         name='urine_strip_train'
#     )
    
#     print("=== 모델 학습 완료 ===")
#     return model

# # 학습 실행 (필요시 파라미터 조정)
# model = train_yolo_model(epochs=100, batch=4)  # 테스트용으로 적은 epoch

=== YOLO 모델 학습 시작 ===
New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.196 🚀 Python-3.10.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=urine_strip_train,

In [9]:
def predict_and_save_labels(model_path='runs/detect2/urine_strip_train/weights/best.pt'):
    """학습된 모델로 추론하고 YOLO 형식 라벨 저장"""
    print("=== 추론 및 라벨 생성 시작 ===")
    
    # 최적 모델 로드
    best_model = YOLO(model_path)
    
    # 추론할 이미지 목록
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    inference_images = []
    
    for ext in image_extensions:
        inference_images.extend(glob.glob(f'inference_images/{ext}'))
        inference_images.extend(glob.glob(f'inference_images/{ext.upper()}'))
    
    print(f"추론할 이미지: {len(inference_images)}개")
    
    for img_path in inference_images:
        img_name = os.path.basename(img_path)
        base_name = os.path.splitext(img_name)[0]
        
        # 추론 실행 (더 많은 검출 허용)
        results = best_model.predict(
            source=img_path,
            conf=0.1,  # confidence 낮춰서 더 많이 검출
            iou=0.5,   # NMS IoU threshold 낮춰서 중복 허용
            max_det=300 # 충분히 많이 검출
        )
        
        # 정확히 5개 요검사지 + 55개 패드 선별
        filtered_boxes = filter_exact_60_objects_with_nms(results[0], iou_threshold=0.2)
        
        # YOLO 형식으로 라벨 저장
        save_filtered_labels(filtered_boxes, f'have_to_train_txt/{base_name}.txt')
        
        print(f"처리 완료: {img_name} - 검출된 객체: {len(filtered_boxes)}개")
    
    print("=== 추론 및 라벨 생성 완료 ===")

In [10]:
def calculate_iou(box1, box2):
    """
    두 바운딩 박스의 IoU(Intersection over Union)를 계산합니다.
    box 형식: {'x_center': ..., 'y_center': ..., 'width': ..., 'height': ...}
    """
    # 박스 1의 좌표 (x_min, y_min, x_max, y_max)
    x1_min = box1['x_center'] - box1['width'] / 2
    y1_min = box1['y_center'] - box1['height'] / 2
    x1_max = box1['x_center'] + box1['width'] / 2
    y1_max = box1['y_center'] + box1['height'] / 2
    area1 = box1['width'] * box1['height']

    # 박스 2의 좌표
    x2_min = box2['x_center'] - box2['width'] / 2
    y2_min = box2['y_center'] - box2['height'] / 2
    x2_max = box2['x_center'] + box2['width'] / 2
    y2_max = box2['y_center'] + box2['height'] / 2
    area2 = box2['width'] * box2['height']

    # 겹치는 영역(intersection)의 좌표
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    # 겹치는 영역의 너비와 높이
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)

    # 겹치는 영역의 넓이
    intersection_area = inter_width * inter_height

    # 합집합(union) 영역의 넓이
    union_area = area1 + area2 - intersection_area

    # IoU 계산
    iou = intersection_area / union_area if union_area > 0 else 0
    return iou


# IoU 계산 함수는 이 위에 있어야 합니다.

def filter_exact_60_objects_with_nms(result, iou_threshold=0.5):
    """
    NMS를 적용하여 중복을 제거한 후,
    정확히 클래스0 5개, 클래스1 55개 선별
    """
    
    def apply_custom_nms(detections, threshold):
        """탐지된 객체 리스트에 NMS를 적용하는 내부 함수"""
        if not detections:
            return []

        # confidence 점수 순으로 정렬
        detections_sorted = sorted(detections, key=lambda x: x['conf'], reverse=True)
        
        kept_detections = []
        while detections_sorted:
            # 가장 점수가 높은 박스를 선택
            best_det = detections_sorted.pop(0)
            kept_detections.append(best_det)
            
            # 나머지 박스들과 IoU 비교
            remaining_detections = []
            for det in detections_sorted:
                iou = calculate_iou(best_det, det)
                # IoU가 threshold 미만인, 즉 겹치지 않는 박스만 남김
                if iou < threshold:
                    remaining_detections.append(det)
            
            detections_sorted = remaining_detections
            
        return kept_detections

    if result.boxes is None:
        return []

    # 1. 모든 탐지 결과를 리스트로 변환
    all_detections = []
    for i, box in enumerate(result.boxes):
        all_detections.append({
            'cls': int(box.cls[0]), 'conf': float(box.conf[0]),
            'x_center': float(box.xywhn[0][0]), 'y_center': float(box.xywhn[0][1]),
            'width': float(box.xywhn[0][2]), 'height': float(box.xywhn[0][3]),
        })

    # 2. 클래스별로 분리
    strips_raw = [det for det in all_detections if det['cls'] == 0]
    pads_raw = [det for det in all_detections if det['cls'] == 1]
    
    # 3. 각 클래스에 NMS 적용하여 중복 제거 (!!! 여기가 핵심 !!!)
    strips_unique = apply_custom_nms(strips_raw, iou_threshold)
    pads_unique = apply_custom_nms(pads_raw, iou_threshold)
    
    # 4. 중복 제거된 객체들로 나머지 로직 수행
    strips_sorted = sorted(strips_unique, key=lambda x: x['conf'], reverse=True)
    selected_strips = strips_sorted[:5]

    selected_pads = []
    # 패드 후보군을 미리 복사해두어 중복 선택 방지
    remaining_pads = list(pads_unique) 
    
    for strip in selected_strips:
        nearby_pads = []
        for pad in remaining_pads:
            y_diff = abs(pad['y_center'] - strip['y_center'])
            if y_diff < 0.1:
                distance = abs(pad['x_center'] - strip['x_center']) + y_diff
                nearby_pads.append({**pad, 'distance': distance})
        
        nearby_pads_sorted = sorted(nearby_pads, key=lambda x: (x['distance'], -x['conf']))
        strip_pads = nearby_pads_sorted[:11]
        
        selected_pads.extend(strip_pads)
        
        # 이미 선택된 패드는 다음 스트립에서 고려되지 않도록 제거
        for sp in strip_pads:
            # 'distance' 키가 추가되었으므로 원본 딕셔너리로 찾아 제거
            original_pad = {k: v for k, v in sp.items() if k != 'distance'}
            if original_pad in remaining_pads:
                remaining_pads.remove(original_pad)

    final_objects = selected_strips + selected_pads
    return final_objects

In [11]:
def save_filtered_labels(filtered_boxes, save_path):
    """필터링된 객체들을 YOLO 형식으로 저장"""
    with open(save_path, 'w') as f:
        for obj in filtered_boxes:
            f.write(f"{obj['cls']} {obj['x_center']:.6f} {obj['y_center']:.6f} {obj['width']:.6f} {obj['height']:.6f}\n")

# 추론 실행
predict_and_save_labels()

=== 추론 및 라벨 생성 시작 ===
추론할 이미지: 42개

image 1/1 /home/jupyter-seojang0510/_Yolo_Dipstick_training/inference_images/wo Urobilin__PATHSEP__Control__PATHSEP__Z-Flip__PATHSEP__Ctrl_Shadow.jpg: 480x640 5 urine_strips, 55 test_pads, 110.4ms
Speed: 7.2ms preprocess, 110.4ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)
처리 완료: wo Urobilin__PATHSEP__Control__PATHSEP__Z-Flip__PATHSEP__Ctrl_Shadow.jpg - 검출된 객체: 58개

image 1/1 /home/jupyter-seojang0510/_Yolo_Dipstick_training/inference_images/w Urobilin__PATHSEP__Control__PATHSEP__S21+__PATHSEP__Ctrl_4000_100.jpg: 480x640 5 urine_strips, 55 test_pads, 13.0ms
Speed: 20.1ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
처리 완료: w Urobilin__PATHSEP__Control__PATHSEP__S21+__PATHSEP__Ctrl_4000_100.jpg - 검출된 객체: 54개

image 1/1 /home/jupyter-seojang0510/_Yolo_Dipstick_training/inference_images/wo Urobilin__PATHSEP__Control__PATHSEP__Z-Flip__PATHSEP__Ctrl_4000_100.jpg: 480x640 5 urine_strips, 55 test_pa

In [12]:
def visualize_predictions(num_samples=3):
    """추론 결과 시각화"""
    print("=== 결과 시각화 ===")
    
    # 추론 이미지 몇 개 선택
    image_files = glob.glob('inference_images/*.jpg') + glob.glob('inference_images/*.png')
    sample_images = image_files[:num_samples]
    
    fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
    if len(sample_images) == 1:
        axes = [axes]
    
    for idx, img_path in enumerate(sample_images):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 추론 실행
        results = YOLO('runs/detect/urine_strip_train/weights/best.pt').predict(img_path)
        
        # 결과 그리기
        annotated_img = results[0].plot()
        
        axes[idx].imshow(annotated_img)
        axes[idx].set_title(f'Prediction: {os.path.basename(img_path)}')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [13]:
def check_generated_labels():
    """생성된 라벨 파일 확인"""
    label_files = glob.glob('have_to_train_txt/*.txt')
    print(f"생성된 라벨 파일: {len(label_files)}개")
    
    # 첫 번째 라벨 파일 내용 확인
    if label_files:
        with open(label_files[0], 'r') as f:
            content = f.read()
        print(f"\n첫 번째 라벨 파일 내용 ({os.path.basename(label_files[0])}):")
        print(content[:200] + "..." if len(content) > 200 else content)

# 결과 확인
visualize_predictions()
check_generated_labels()

print("\n=== 전체 작업 완료 ===")
print("생성된 파일:")
print("- 학습된 모델: runs/detect/urine_strip_train/weights/best.pt")
print("- 추론 라벨: have_to_train_txt/ 폴더")

=== 결과 시각화 ===


FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/urine_strip_train/weights/best.pt'